<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Multicompartment Simulator**

**Eran Frank Shorer**

**MSc Computational Neuroscience**

**"Investigating the effects of impermeant anions on the electrical and computational properties of neurons"**




## Instructions:

This jupyter notebook consists of cells. Some cells are plain text, while others are code. 

Press Shift-Enter simultaneously, or click the cell and press the play button in the main toolbar above to run the cell. 

The output of the cell should be displayed below the cell. 



## **Import classes and modules** 


In [37]:
from ipywidgets import widgets, Layout, interact, interactive, interactive_output
from IPython.display import display
import common
import constants
import compartment
import electrodiffusion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
print("All relevant classes imported")
!pip install viola



All relevant classes imported


## **Set simulation parameters**


## *A) Compartment parameters*

### A1) Set compartment parameters:

In [40]:
txt_name = widgets.Text(value="Comp1")
lbl_name = widgets.Label(value ="Compartment Name: ")
name_box = widgets.HBox(children = [lbl_name,txt_name])

lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=11.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=100, min=50, max=200, step=20, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

lbl_nai = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nai = widgets.FloatSlider(value=14, min=0, max=100, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_nai = widgets.HBox(children = [lbl_nai,sldr_nai])

lbl_ki = widgets.Label(value = "K+ concentration (mM): ")
sldr_ki = widgets.FloatSlider(value=122, min=0, max=250, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ki = widgets.HBox(children = [lbl_ki,sldr_ki])

lbl_cli = widgets.Label(value = "Cl- concentration (mM): ")
sldr_cli = widgets.FloatSlider(value=5, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_cli = widgets.HBox(children = [lbl_cli,sldr_cli])

lbl_xi = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xi = widgets.FloatSlider(value=150, min=0, max=300, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_xi = widgets.HBox(children = [lbl_xi,sldr_xi])

lbl_zi = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zi = widgets.FloatSlider(value=-0.85, min=-10, max=10, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_zi = widgets.HBox(children = [lbl_zi,sldr_zi])



global name, comp_arr, df_sim


df_sim = pd.DataFrame()

comp_arr =[]
name = txt_name.value
output1 = widgets.Output()


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")


def new_comp_clicked(b):
    
    name = txt_name.value
    for j in range (len(comp_arr)):
        if name == comp_arr[j].name:
            raise Exception("Two compartments cannot have the same name")
    new_comp = compartment.Compartment(name,radius=sldr_rad.value*1e-5, length=sldr_len.value*1e-5)
    new_comp.set_ion_properties(na_i=sldr_nai.value*1e-3, k_i = sldr_ki.value*1e-3, cl_i=sldr_cli.value*1e-3,x_i=sldr_xi.value*1e-3, z_i=sldr_zi.value)
    comp_arr.append(new_comp)
    arr_df = new_comp.get_df_array()
    df_sim[name] = arr_df
    df_sim.index = ['Radius', 'Length', 'Volume', 'Na_i', 'K_i', 'Cl_i', 'X_i', 'z_i', 'Vm', 'Ek', 'ECl', 'ATPase pump rate',
      'KCC2 pump rate']
    with output1:
        print("Compartment: ("+ new_comp.name +") created" )
        print(new_comp.get_df_array())
      
btn_new_comp.on_click(new_comp_clicked)


vbox_grand = widgets.VBox(children = [name_box,rad_box,len_box, hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi,btn_new_comp], layout=Layout(border ="solid"))

display(output1,vbox_grand)







Output()

Exception: Two compartments cannot have the same name

In [41]:
x = 0
for i in comp_arr:
    print(comp_arr[x].name + ": " + str(comp_arr[x].na_i)+ " Molar")
    x += 1

Comp1: 0.014 Molar
Comp2: 0.014 Molar


### A2) Review & edit compartments

In [51]:
btn_review = widgets.Button(description = "Review compartments", button_style ="success")
output2 = widgets.Output()
display(btn_review,output2)

def btn_review_clicked(b):
    with output2:
        display(df_sim)

btn_review.on_click(btn_review_clicked)       


Button(button_style='success', description='Review compartments', style=ButtonStyle())

Output()

### A3) Multicompartment visualization


In [18]:
btn_visualize = widgets.Button(description = "Visualize compartments")


tog_params = widgets.ToggleButtons(
    options=['Radius', 'Length', 'Na+ Conc','K+ Conc','Cl- Conc', 'Imperm. Conc', 'Imperm. Charge'],
    description='',
    disabled=False,
    button_style='info',
    # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 3
)
tog_params.style.button_width ='120px'
display(tog_params)

ii= 0
for i in comp_arr:
    hgt = str(round(comp_arr[ii].length*80000))+"px"
    wdth = str(round(comp_arr[ii].radius*5000000))+"px"
    btn = widgets.Button(layout=Layout(height = hgt, width =wdth,border="solid", disabled ="true"), description = comp_arr[ii].name)
    btn.style.button_color = 'green'
    display(btn)
    ii +=1



#btn1 = widgets.Button(layout=Layout(height = hgt, width ="50px",border="solid", disabled ="true"))
#lbl = widgets.Label(value = "100")
#btn1.style.button_color = 'red'
#btn_arr.append(btn1)

#btn2 = widgets.Button(layout=Layout(height = "40px", width ="70px",border="solid",disabled ="true"))
#lbl = widgets.Label(value = "100")
#btn2.style.button_color = 'blue'
#btn_arr.append(btn2)





ToggleButtons(button_style='info', options=('Radius', 'Length', 'Na+ Conc', 'K+ Conc', 'Cl- Conc', 'Imperm. Co…

Button(description='Comp1', layout=Layout(border='solid', height='80px', width='250px'), style=ButtonStyle(but…

Button(description='Comp2', layout=Layout(border='solid', height='80px', width='250px'), style=ButtonStyle(but…

### A4) Electrodiffusion setup

In [ ]:
## Need to code the electrodiffusion steps here

In [17]:
## Loop to run through the compartment array and join them by electrodiffusion
ed_array = []

for e in range(len(comp_arr)-1):
    ed_array.append(electrodiffusion.Electrodiffusion(comp_arr[e],comp_arr[e+1]))
    print

    

IndexError: list index out of range

## *B) Timing*

In [8]:
lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")
sldr_total_t = widgets.FloatSlider(value=3, min=1, max=20, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_total_t = widgets.HBox(children = [lbl_total_t,sldr_total_t])

lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_dt = widgets.HBox(children = [lbl_dt,sldr_dt])

dt= sldr_dt.value*1e-3 # 1ms time steps 
total_t= sldr_total_t.value *60 #s

run_t = 0 #current simulation timing
t_arr = [0]


vbox_t = widgets.VBox([HBox_total_t,HBox_dt], layout=Layout(border='solid'))

btn_sim = widgets.Button(description ="Run Simulation", button_style="success")



display(vbox_t,btn_sim)

#print("Set total simulation time (minutes):")

#print("Set time step (milliseconds):")


Button(button_style='success', description='Run Simulation', style=ButtonStyle())

## *C) Run Simulation*

In [13]:
while run_t < total_t:

    
    # will need to loop through comp_arr to update all these values
    
    
    comp_1.step(dt)
    comp_2.step(dt)
    comp_1_ed_dict = comp_1.get_ed_dict()
    comp_2_ed_dict = comp_2.get_ed_dict()
    ed_conc_changes = ed_1_2.calc_ed(dt,comp_1_ed_dict,comp_2_ed_dict)
    comp_1.ed_update(ed_conc_changes)
    for j in ed_conc_changes:
        ed_conc_changes[j] *= -1
    comp_2.ed_update(ed_conc_changes)
    comp_1.update_volumes()
    comp_1.update_arrays()
    run_t += dt
    t_arr.append(run_t)
    df_sim[comp_1.name]=comp_1.get_df_array()
    df_sim[comp_2.name]=comp_2.get_df_array()



TypeError: 'dict_keys' object is not subscriptable